In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import wcs
from astropy.io import ascii
from astropy.table import Table, Column, MaskedColumn
from pathlib import Path
from math import trunc
from scipy import optimize
import copy
%matplotlib inline

SMALL_SIZE = 10*2                                        
MEDIUM_SIZE = 12*2
BIGGER_SIZE = 14*2

plt.rc('font', size=SMALL_SIZE, family='serif')          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)                     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)                    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE, direction='in')    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE, direction='in')    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)                    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)                  # fontsize of the figure title
plt.rc('figure', figsize='8, 6')                         # size of the figure, used to be '4, 3' in inches

In [ ]:
def hulst(a,wl):
    n = n_silicate
    p = 4*np.pi*a*(n-1)/wl
    return 2 - ((4/p)*np.sin(p)) + (4/(p**2)*(1-np.cos(p)))

def num_int(h, w, trans):
    return np.sum(h*w*trans)

def Nparticles(dmag, A, a, Q):
    return (1/(np.pi*(a**2)*Q)) * (1-(10**(dmag/(-2.5))))

def ref_index(wl):
    return np.sqrt(1+ ((0.6961663*wl**2)/(wl**2 - (0.0684043)**2)) + ((0.4079426*wl**2)/(wl**2 - (0.1162414)**2)) +\
                   ((0.8974794*wl**2)/(wl**2 - (9.896161)**2)))

filt = ['gp','rp','ip']
filt_c = {'gp':'g','rp':'r','ip':'indianred'}
filt_trans = {}
for f in filt:
    d = ascii.read("filter_transmissions/SDSS."+f+".txt")
    filt_trans[f] = np.array([d["\lambda"],d["[nm]"]])
    filt_trans[f][0] *= 1e-9 #from nm to m

gaiabp = ascii.read('filter_transmissions/GaiaDR2_RevisedPassbands.dat')
gaiabpc = {2:'g',4:'b',6:'r'}
for f in filt:
    plt.plot(filt_trans[f][0]*1e9,filt_trans[f][1],c=filt_c[f])
for c in [2,4,6]:
    msk = gaiabp['col'+str(c)] != 99.99
    plt.errorbar(gaiabp['col1'][msk],gaiabp['col'+str(c)][msk],yerr=gaiabp['col'+str(c+1)][msk],fmt='--.',c=gaiabpc[c])
plt.show()


n_silicate = ref_index(6215e-4) #middle of rp filter
print ("Refractive index silicate = {:.5f}".format(n_silicate))

In [ ]:
wlarr = np.arange(0.21,1.71,0.001)*1e-6
dsize = 1e-7

plt.figure(figsize=(12,4))
plt.plot(wlarr, hulst(dsize,wlarr))
for f in filt:
    plt.plot(filt_trans[f][0],filt_trans[f][1],c=filt_c[f])
plt.xscale('log')
plt.show()
plt.figure(figsize=(12,4))
plt.plot(wlarr, ref_index(wlarr*1e6),marker='.')
for f in filt:
    plt.plot(filt_trans[f][0],filt_trans[f][1],c=filt_c[f])
#plt.xscale('log')
plt.show()

In [ ]:
wl_arr = np.arange(200,1000,0.1) * 1e-9 # m
#wl_arr = np.arange(400,860,0.1) * 1e-9 # m

#sizes = np.arange(1,20,2) * 1e-8 #m
#sizes = [20e-8]
sizes = np.logspace(1e-9,3e-7,10000)-1
#sizes = [5e-1, 5e-2, 5e-3,5e-4,5e-5,5e-6,5e-7,5e-8]

#fig, axs = plt.subplots(len(sizes), sharex=True, gridspec_kw={'hspace': 0})
#fig.set_figheight(3*len(sizes))
#fig.set_figwidth(16)
#for i in range(len(sizes)):
#    axs[i].plot(wl_arr,hulst(sizes[i],wl_arr))
#    for f in filt:
#        axs[i].plot(filt_trans[f][0],filt_trans[f][1],c=filt_c[f])
#    axs[i].set_ylim(0,3.5)
#    axs[i].set_yscale('log')
#    axs[i].set_ylim(1e-1,3.5)
#axs[i].set_xscale('log')
#axs[i].set_xlim(np.min(wl_arr),np.max(wl_arr))
#axs[i].invert_xaxis()

# Hide x labels and tick labels for all but bottom plot.
#for ax in axs:
#    ax.label_outer()
#plt.show()

# plt.figure(figsize=(8,5))
# for s in sizes:
#     plt.plot(wl_arr,hulst(s,wl_arr),label="Dust with radius $0.2\mu m$",c='b')
#     plt.ylim(0,3.5)
# #    axs[i].set_yscale('log')
# #    axs[i].set_ylim(1e-1,3.5)
# for f in filt:
#     plt.plot(filt_trans[f][0],filt_trans[f][1],c=filt_c[f],label=f+" bandpass")
# plt.xscale('log')
# plt.xlim(np.min(wl_arr),np.max(wl_arr))
# plt.axis(ymin=0.01,ymax=3.25,xmin=2e-8,xmax=1e-6)
# plt.legend(loc=4)
# plt.ylabel("$Q_t$")
# plt.xlabel("Wavelength ($nm$)")
# plt.title("Anomalous diffraction with LCOGT filters")
# plt.gca().invert_xaxis()
# plt.show()

# plt.figure(figsize=(16,8))
# for s in sizes:
#     plt.plot(wl_arr,hulst(s,wl_arr))
#     plt.ylim(0,3.5)
# #    axs[i].set_yscale('log')
# #    axs[i].set_ylim(1e-1,3.5)
# for f in filt:
#     plt.plot(filt_trans[f][0],filt_trans[f][1],c=filt_c[f])
# plt.xscale('log')
# plt.xlim(np.max(wl_arr),np.min(wl_arr))
# plt.show()

In [ ]:
def mie_ext_func(a,f):
    lamb = (filt_trans[f][0][1:]+filt_trans[f][0][:-1])/2
    dlamb = (filt_trans[f][0][1:]-filt_trans[f][0][:-1])
    trans = (filt_trans[f][1][1:]+filt_trans[f][1][:-1])/2
    return np.sum(hulst(a,lamb)*dlamb*trans) / np.sum(dlamb*trans)

mie_ext = {}
for f in filt:
    tmp = []
    for s in sizes:
        tmp.append(mie_ext_func(s,f))
    mie_ext[f] = np.array(tmp)
    

In [ ]:
plt.figure(figsize=(10,5))
for f in filt:
    plt.plot(1e6*sizes,mie_ext[f],marker='.',c=filt_c[f],label=f)
plt.xlabel("Dust radius (microns)")
plt.ylabel("$Q_{t,f}$")
plt.title("Mie-scattering coefficient against dust-size")
#plt.xscale('log')
plt.axis(ymin=0,xmin=sizes[0]*1e6,xmax=sizes[-1]*1e6)
plt.legend()
plt.show()

In [ ]:
#insert J0600 lightcurve
epoch = {}
mag = {}
magerr = {}
for f in filt:
    d = ascii.read('J0600_mag/LCOGT_'+f+'.txt')
    epoch[f] = np.array(d["EPOCH"])
    mag[f] = np.array(d["NORMMAG"])
    magerr[f] = np.array(d["MAGERR"])

Ndays_bin = 5

num, bins = np.histogram(epoch['ip'],np.int((epoch['ip'][-1]-epoch['ip'][0])/Ndays_bin))
plt.plot((bins[1:]+bins[:-1])/2,num)
plt.show()
print (len(bins))

print ((bins))

# e_lim = [0,50,100,150,212]
# for beg, end in zip(e_lim[:-1], e_lim[1:]):
#     plt.figure(figsize=(16,6))
#     plt.plot([0,1e6],[0,0],linestyle='--',c='black')
#     for f in filt:
#         plt.errorbar(epoch[f],mag[f],yerr=magerr[f],fmt='.',c=filt_c[f])

#     for e in bins:
#         plt.plot([e,e],[0,.5],c='black',linewidth=1)
#     plt.xlim(bins[beg]-2,2+bins[end])
#     #plt.xlim(e1-5,5+e2)
#     plt.ylim(-0.05,0.75)
#     plt.gca().invert_yaxis()
#     plt.show()

In [ ]:
# Select epochs for interesting parts of the lightcurve\
point = {}
for f in filt:
    tmp = []
    for i in range(len(bins)-1):
        e1, e2 = bins[i:i+2]
    
        m1 = epoch[f]>e1
        m2 = epoch[f]<e2
        if np.sum(m1*m2) != 0:
            #w = 1/magerr[f][m1*m2]
            #tmp.append(np.sum(mag[f][m1*m2]*w)/np.sum(w))
            tmp.append(np.nanmedian(mag[f][m1*m2]))
        else:
            tmp.append(np.nan)
    point[f] = np.array(tmp)

In [ ]:
plt.figure(figsize=(12,6))
for i in range(len(bins)-2):
    e1, e2 = bins[i:i+2]
    
    for f in filt:
        plt.plot([e1,e2],[point[f][i]]*2,c=filt_c[f])


    plt.plot([0,1e6],[0,0],linestyle='--',c='black')
    
    for e in [e1,e2]:
        plt.plot([e,e],[0,.5],c='black')
        
for f in filt:
    plt.errorbar(epoch[f],mag[f],yerr=magerr[f],fmt='.',c=filt_c[f],label='LCOGT '+f)
plt.xlim(np.min(epoch['gp'])-2,2+np.max(epoch['gp']))
#plt.xlim(e1-5,5+e2)
plt.title("J0600 binned magnitude drop")
plt.ylabel('$\Delta mag$')
plt.xlabel('Epoch [MJD]')
plt.xlim(59163.4,59193.9)
plt.ylim(-0.01,0.4)
plt.gca().invert_yaxis()
plt.legend(loc=4)
plt.show()

In [ ]:
def Nparticles(dmag, a, Q):
    return (1/(np.pi*(a**2)*Q)) * (1-(10**(dmag/(-2.5)))) 

Npar = {}
for f in filt:
    Npar[f] = []

for i in range(len(num)):
    for f in filt:
        Npar[f].append(Nparticles(point[f][i],sizes,mie_ext[f]))
for f in filt:
    Npar[f] = np.array(Npar[f])

In [ ]:
# for i in range(len(num)):
#     plt.figure(figsize=(8,6))
#     for f in filt:
#         plt.plot(sizes,Npar[f][i],marker='.',c=filt_c[f])
#     plt.yscale('log')
#     plt.xscale('log')
#     plt.show()

In [ ]:
ep = (bins[1:] + bins[:-1])/2

dsize1 = []
dsizeerr1 = []
dsize2 = []
NP = []
NPerr = []
NP2 = []

for i in range(len(num)):
    tmp = []
    tmp2 = []
    for f1, f2 in zip(['gp','rp','ip'],['rp','ip','gp']):
    #for f1, f2 in zip(['gp'],['ip']):
        dis = np.abs(Npar[f1][i] - Npar[f2][i])
        if np.sum(np.isnan(dis)) == 0:
            i_max = np.where(dis == np.min(dis))[0]

            #plt.plot([1e6*sizes[i_max]]*2, [0,-1e-7],c='black')
            #plt.plot(1e6*sizes,dis,marker='.',label=f1+'-'+f2)
            
            tmp.append(sizes[i_max])
            tmp2.append((Npar[f1][i]+Npar[f2][i])[i_max]/2)
    dsize1.append(np.mean(tmp))
    dsize2.append(np.array(tmp))
    dsizeerr1.append(np.std(tmp))
    NP.append(np.mean(tmp2))
    NPerr.append(np.std(tmp2))
    NP2.append(Nparticles(point['gp'][i],dsize1[i],mie_ext_func(dsize1[i],'gp')))
            

    #plt.xlabel("Dust size (micrometer)")
    #plt.ylabel("J0600_mag/mie_ext")
    #plt.ylim(-1e-7,2e-7)
    #plt.legend()
    #plt.show()

dsize1 = np.array(dsize1)
dsizeerr1 = np.array(dsizeerr1)
NP = np.array(NP)
NPerr = np.array(NPerr)
NP2 = np.array(NP2)
#print (NP2)
print (np.shape(NP2))
print (np.shape(ep))
print (np.shape(dsizeerr1 != 0))

In [ ]:
# print (len(num))
# for i in np.arange(len(num))[20:]:
#     if np.isnan(Npar['gp'][i][0]) == False:
#         plt.figure(figsize=(8,4))
#         for f1, f2 in zip(['gp','rp','ip'],['rp','ip','gp']):
#             dis = np.abs(Npar[f1][i] - Npar[f2][i])
#             if np.sum(np.isnan(dis)) == 0:
#                 i_max = np.where(dis == np.min(dis))[0]
#                 plt.plot([1e6*sizes[i_max]]*2, [8e10,5e11],c='black')
#                 #plt.plot(1e6*sizes,dis,label=f1+'-'+f2)
#         for f in filt:
#             if np.sum(np.isnan(Npar[f][i])) == 0:
#                 plt.plot(1e6*sizes,Npar[f][i],label=f,c=filt_c[f])

#         plt.xlabel("Dust radius (micrometer)")
#         plt.ylabel("N particles")
#         plt.yscale('log')
#         plt.ylim(1e11,5e13)
#         plt.xlim(0.08,0.4)
#         plt.title("Dust size calculation demonstration")
#         plt.legend()
#         plt.show()

# for i in np.arange(len(num))[20:]:
#     if np.isnan(Npar['gp'][i][0]) == False:
#         plt.figure(figsize=(12,12))
#         for f1, f2 in zip(['gp','rp','ip'],['rp','ip','gp']):
#             dis = np.abs(Npar[f1][i] - Npar[f2][i])
#             if np.sum(np.isnan(dis)) == 0:
#                 i_max = np.where(dis == np.min(dis))[0]
#                 plt.plot([1e6*sizes[i_max]]*2, [1e12,1e15],c='black')
#                 plt.plot(1e6*sizes,dis,label=f1+'-'+f2)
#         #for f in filt:
#         #    if np.sum(np.isnan(Npar[f][0])) == 0:
#         #        plt.plot(1e6*sizes,Npar[f][i],label=f,c=filt_c[f])

#         plt.xlabel("Dust size (micrometer)")
#         plt.ylabel("J0600_mag/mie_ext")
#         plt.yscale('log')
#         #plt.ylim(0,2e-6)
#         #plt.xlim(0,0.2)
#         plt.title(ep[i])
#         plt.legend()
#         plt.show()


In [ ]:
mask = dsizeerr1 != 0

fig, axs = plt.subplots(3, sharex=True, gridspec_kw={'hspace': 0})
#fig.title('Magnitude of J0600 in gp and rp filter ')
fig.set_figheight(12)
fig.set_figwidth(16)
#for s in [sizes[0],sizes[-1]]:
#    axs[0].plot([ep[0],ep[-1]],[s*1e6]*2,c='black',linestyle='--')

axs[0].errorbar(ep[mask], 1e6*dsize1[mask], yerr=1e6*dsizeerr1[mask],fmt='o',c='b',linewidth=1,label='3 filters')
#for e in np.arange(len(ep))[mask]:
#    if len(dsize2[e]) != 0:
#        axs[0].scatter([ep[e]]*3,dsize2[e]*1e6,c='black',marker='.',s=10)
axs[0].scatter(ep[mask==False], 1e6*dsize1[mask==False],marker='o',c='r',label='2 filters')
axs[0].set_ylabel("Dust radius $(\mu m)$")
axs[0].legend()
axs[0].plot([0,1e8],[0.28]*2)
axs[0].plot([0,1e8],[0.36]*2)
axs[0].grid()

axs[1].errorbar(ep[mask], NP[mask], yerr=NPerr[mask],fmt='o',c='b',linewidth=1,label='3 filters')
axs[1].scatter(ep[mask==False], NP[mask==False],marker='o',c='r',label='2 filters')
axs[1].set_ylabel("N particles")
axs[1].set_ylim(ymin=1e10,ymax=1.06e12)
axs[1].legend()
axs[1].grid()

axs[2].plot([epoch['ip'][0],epoch['ip'][-1]],[0,0],c='black',linestyle='--')
for f in filt:
    axs[2].errorbar(epoch[f],mag[f],yerr=magerr[f],fmt='.',c=filt_c[f],label='LCOGT '+f)
axs[2].invert_yaxis()
axs[2].set_ylabel("J0600 $\Delta$m")
axs[2].legend()
axs[2].grid()

axs[2].set_xlim([epoch['ip'][0]-2,2+epoch['ip'][-1]])
axs[2].set_xlabel('Epoch [MJD]')
axs[0].set_title("J0600 dust radius and number of particles throughout its eclipse")
plt.savefig("plots/J0600_dustsize.jpg",dpi=400)
plt.show()

plt.figure(figsize=(16,8))
for f in filt:
    plt.plot(epoch[f],mag[f],linewidth=0.5,c=filt_c[f])
    for i in range(len(dsize1)):
        plt.scatter(ep[i], -2.5*np.log10(1-(np.pi*dsize1[i]**2*NP[i]*mie_ext_func(dsize1[i],f))),c=filt_c[f])
plt.gca().invert_yaxis()
plt.xlim([epoch['ip'][0]-2,2+epoch['ip'][-1]])
plt.show()